In [ ]:
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain.document_loaders import WebBaseLoader
from langchain.agents import initialize_agent, Tool
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
import os


wikipedia_tool = Tool(
    name="Wikipedia",
    func=WikipediaQueryRun().run,
    description="Use this tool to search for information on Wikipedia."
)


duckduckgo_tool = Tool(
    name="DuckDuckGo",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to search for information on DuckDuckGo."
)


def scrape_website(url):
    loader = WebBaseLoader(url)
    documents = loader.load()
    return "\n\n".join([doc.page_content for doc in documents])

web_scraper_tool = Tool(
    name="WebScraper",
    func=scrape_website,
    description="Use this tool to scrape content from a website given its URL."
)


def save_to_txt_file(content, filename="research_results.txt"):
    with open(filename, "w", encoding="utf-8") as file:
        file.write(content)
    return f"Results saved to {filename}"

save_tool = Tool(
    name="SaveToTxt",
    func=lambda content: save_to_txt_file(content),
    description="Use this tool to save content to a .txt file."
)


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = [wikipedia_tool, duckduckgo_tool, web_scraper_tool, save_tool]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


query = "Research about the XZ backdoor"
result = agent.run(query)
print("\nResearch complete!")
